In [1]:
import numpy as np
import matplotlib.pyplot as plt

In [3]:
m = 100000

x = np.random.normal(size=m)
X = x.reshape(-1, 1)
y = 4.*x + 3. + np.random.normal(0, 3, size=m)

In [6]:
def J(theta, X_b, y):
    try:
        return np.sum((y - X_b.dot(theta))**2) / len(X_b)
    except:
        return float("inf")

def dJ(theta, X_b, y):
    return X_b.T.dot(X_b.dot(theta) - y) * 2. /len(y)

def gradient_descent(X_b, y, initial_theta, eta, n_iters=1e4, epsilon = 0.01):

    theta = initial_theta
    cur_iter = 0
    
    while cur_iter < n_iters:
        gradient = dJ(theta, X_b, y)
        last_theta = theta
        theta = theta- eta* gradient
        if(abs(J(theta, X_b, y)-J(last_theta, X_b, y))< epsilon):
            break
            
        cur_iter += 1
        
    return theta
        

In [7]:
%%time 
X_b = np.hstack([np.ones((len(X), 1)), X])
initial_theta = np.zeros(X_b.shape[1])
eta = 0.01
theta =gradient_descent(X_b, y, initial_theta, eta)

Wall time: 915 ms


In [8]:
theta

array([2.72352251, 3.59880937])

### 随机梯度下降法

In [9]:
def dJ_sgd(theta, X_b_i, y_i):
    return X_b_i.T.dot(X_b_i.dot(theta) - y_i) * 2. 

def sgd(X_b, y, initial_theta, n_iters=1e4):
    t0 = 5
    t1 = 50
    
    def learning_rate(t):
        return t0/(t+t1)
    
    theta = initial_theta
    
    for cur_iter in range(n_iters):
        rand_i = np.random.randint(len(X_b))
        
        gradient = dJ_sgd(theta, X_b[rand_i], y[rand_i])
        theta = theta- learning_rate(cur_iter)* gradient     
        
    return theta
        

In [10]:
%%time 
X_b = np.hstack([np.ones((len(X), 1)), X])
initial_theta = np.zeros(X_b.shape[1])
theta = sgd(X_b, y, initial_theta, n_iters=len(X_b) // 3)

Wall time: 391 ms


In [11]:
theta

array([3.02780773, 3.96887109])

### 波士顿房价——使用真实数据

In [11]:
from sklearn import datasets
import numpy as np
import matplotlib.pyplot as plt
boston = datasets.load_boston()
X = boston.data
y = boston.target

X = X[y<50.0]
y = y[y<50.0]

In [12]:

from playML.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, seed=666)

In [13]:
from sklearn.preprocessing import StandardScaler

stdScaler = StandardScaler()
stdScaler.fit(X_train)
X_train_std = stdScaler.transform(X_train)
X_test_std = stdScaler.transform(X_test)

In [14]:
from playML.LinearRegression import LinearRegression
lin_reg = LinearRegression()
%time lin_reg.fit_sgd(X_train_std, y_train, n_iters = 2)
lin_reg.score(X_test_std, y_test)

Wall time: 16.1 ms


-2.090403312891986

In [15]:
%time lin_reg.fit_sgd(X_train_std, y_train, n_iters = 50)
lin_reg.score(X_test_std, y_test)

Wall time: 352 ms


0.8129786089726068

In [16]:
%time lin_reg.fit_sgd(X_train_std, y_train, n_iters = 100)
lin_reg.score(X_test_std, y_test)

Wall time: 430 ms


0.8131536586128989

### sklearn中的sgd

In [17]:
from sklearn.linear_model import SGDRegressor
sgd_reg = SGDRegressor()
%time sgd_reg.fit(X_train_std, y_train)
sgd_reg.score(X_test_std, y_test)

Wall time: 8.6 ms


C:\Users\Amo\Anaconda3\lib\site-packages\sklearn\linear_model\stochastic_gradient.py:128: FutureWarning: max_iter and tol parameters have been added in <class 'sklearn.linear_model.stochastic_gradient.SGDRegressor'> in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  "and default tol will be 1e-3." % type(self), FutureWarning)


0.8047845970157302